In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
import os

##### Spark

In [2]:
LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

conf = SparkConf() \
    .setAppName("Acoes") \
    .setSparkHome("./spark/home") \
    .set("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY") \
    .set("spark.sql.legacy.parquet.int96RebaseModeInWrite", "LEGACY") \
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .set("spark.sql.debug.maxToStringFields", "100") \
    .set("spark.sql.warehouse.dir", LAKE_HOME) \
    .set("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .set("spark.driver.extraClassPath", "/usr/local/etc/jdbc/ojdbc10.jar:/usr/local/etc/jdbc/postgresql-42.6.0.jar") \
    .set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d00d0267-2889-446d-b122-b7d37da50bd2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 278ms :: artifacts dl 4ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runt

##### Data Warehouse

In [3]:
WAREHOUSE_TYPE = os.getenv("WAREHOUSE_TYPE", "jdbc")
WAREHOUSE_URI = os.getenv("WAREHOUSE_URI")
WAREHOUSE_DATABASE = os.getenv("WAREHOUSE_DATABASE", "dw")
WAREHOUSE_USER = os.getenv("WAREHOUSE_USER", "dw")
WAREHOUSE_PASSWORD = os.getenv("WAREHOUSE_PASSWORD", "dw")

##### Functions

##### DW read and write

In [4]:
def write_to_dw(df, table_name: str):
    df.write \
        .format("jdbc") \
        .option("url", WAREHOUSE_URI) \
        .option("database", WAREHOUSE_DATABASE) \
        .option("user", WAREHOUSE_USER) \
        .option("password", WAREHOUSE_PASSWORD) \
        .option("dbtable", table_name) \
        .mode("overwrite") \
        .save()
    

def read_from_dw(spark, table_name: str):
    return spark.read \
        .format("jdbc") \
        .option("url", WAREHOUSE_URI) \
        .option("query", "select * from {}".format(table_name)) \
        .option("user", WAREHOUSE_USER) \
        .option("password", WAREHOUSE_PASSWORD) \
        .option("statement_timeout", 480000) \
        .load()

##### Other functions

In [5]:
@udf
def name_adjust(ticker):
    if ticker != '^BVSP':
        r = ticker.removesuffix('.SA')
    else:
        r = 'IBOV'

    return r

NameError: name 'udf' is not defined